In [1]:
import requests, bs4, time
import pandas as pd
import os.path

In [2]:
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)

First searching for all job postings using the word 'Data' on www.monster.co.uk

In [ ]:
url = 'https://www.monster.co.uk/jobs/search/?q=data&cy=uk&client=power&stpage=1&page=10'

In [ ]:
#  GET request
res = requests.get(url)
soup = bs4.BeautifulSoup(res.content)

Extracting all the links to the individual job postings. The links that contain 'job-openings' are only working, so select only those links:

In [ ]:
def extract_links(soup):
    links =[]
    for div in soup.find_all(name='a', attrs={'data-bypass':'true'}):
        links.append(div['href'])
    return links

all_links = extract_links(soup)
links = [s for s in all_links if "job-openings" in s]

Write extracted links into a txt-File.

In [ ]:
with open(parent_folder+'/data/monster_links_1.txt', 'w') as f:
    for item in links:
        f.write("%s\n" % item)

# Scraping websites for each individual job posting link:
First extracting the links from the text file.

In [4]:
links = [line.rstrip('\n') for line in open(parent_folder+'/data/monster_links_1.txt','r')]

Function for extracting the job title from the website:

In [5]:
def extract_jobtitle(soup):
    title = 'no_title'
    for div in soup.find_all(name='h1', attrs={'class':'title'}):
        title = div.text
    return title

Function for extracting the job description from the website:

In [6]:
def extract_descr(soup):
    jobdesc = []
    for div in soup.find_all(name='span', attrs={'id':'TrackingJobBody'}):
        jobdesc = div.text
    return jobdesc

Function for extracting salary from the website:

In [7]:
def extract_salary(soup):
    salary = []
    for div in soup.find_all('div',{'class':'col-xs-12 cell'}):
        salary = div.text
        salary = salary.replace('Salary','').strip()
    return salary

Function for extracting meta-data from the website:

In [9]:
def extract_summary(soup):
    output = ['nan','nan','nan','nan','nan','nan','nan']
    separation =['Location', 'Job type', 'Posted', 'Industries', 'Education level', 'Career level','Reference code']
    for div in soup.find_all('dl',{'class':'header'}):
        summary = div.text
        summary = summary.replace('\n','')
        for idx, item in enumerate(separation):
            if item in summary:
                output[idx] = summary.replace(item,'')
    return output

In [12]:
job_title_company = []
description = []
salary = []
location = []
jobtype = []
industry = []
education = []
career = []
duration = []
ref_code = []

for i in range(0,len(links)):
    url = links[i]
    res = requests.get(url)
    time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = bs4.BeautifulSoup(res.content)
    job_title_company.append(extract_jobtitle(soup))
    description.append(extract_descr(soup))
    salary.append(extract_salary(soup))
    location_new, jobtype_new, duration_new, industry_new, education_new, career_new, ref_code_new = extract_summary(soup)
    location.append(location_new)
    jobtype.append(jobtype_new)
    duration.append(duration_new)
    industry.append(industry_new)
    education.append(education_new)
    career.append(career_new)
    ref_code.append(ref_code_new)
    
d = {'title':job_title_company,'description':description, 'salary':salary, 'location':location, 'jobtype':jobtype, 'duration':duration, 'industry':industry, 'education':education, 'career':career,'ref_code':ref_code}
df = pd.DataFrame(d)
df.to_csv(parent_folder+'/data/monster_links_1.csv', index=True, sep='\t')

In [13]:
df

,title,description,salary,location,jobtype,duration,industry,education,career,ref_code
0,Data Visualisation Developer - ADLIB,Create novel ways of visualising data.Develop ...,"£30,000.00 - £60,000.00 per year Benefits:","Bath, SW BA11SU","Full Time, Permanent",2 Days ago,Computer/IT Services,nan,Entry Level,11083
1,Graduate Sales Trainee - Data Communications -...,STEM graduates needed for an immediate start J...,"£20,000.00 - £22,000.00 per year£10,000 Bonus...","Birmingham, MID B11AA","Full Time, Permanent",2 Days ago,Other/Not Classified,Unspecified,nan,jn -022019-148968_1570441099
2,"Consulting Director, Data - TOMGANDHI CONSULTI...","Consulting Director, Data LondonTo Salary plu...",[],"London, London","Full Time, Permanent",2 Days ago,Computer/IT Services,nan,nan,nan
3,Data Analyst - Verastar Ltd,Winner of Business of the Year at the Greater ...,"£20,000.00 - £22,000.00 per year","Manchester, NW M224DJ","Full Time, Permanent",2 Days ago,Other/Not Classified,Professional,Entry Level,473847
4,Data Architect - Design/Analysis - Central Bir...,Data Architect - Design/Analysis - Central Bir...,"£53,000.00 - £58,000.00 per yearExcellent Ben...","Birmingham, MID B46UD","Full Time, Permanent",2 Days ago,Computer/IT Services,Unspecified,nan,pr/cr/010044_1570463781
...,...,...,...,...,...,...,...,...,...,...
264,Data Processor / Finance Administrator / Data ...,Job Title: Data ProcessorLocation: BanburySala...,"£17,550.00 - £17,550.00 per yearbenefits","Banbury, HC OX150YY","Full Time, Permanent",6 Days ago,Financial Services,nan,nan,bbyfs-60_1570119899
265,Business Analyst (Data Management) - Potters B...,Business Analyst (Business Data Management) - ...,"£50,000.00 - £55,000.00 per year","Hertfordshire, London","Full Time, Permanent",6 Days ago,Computer Software; Computer/IT Services,Unspecified,nan,ash18080ch17_1570088424
266,Data Scientist - Ethero,Expert Level Data AnalystTelford£Excellent sal...,"£41,000.00 - £47,000.00 per year Benefits:","Telford, MID","Full Time, Permanent",1 Days ago,Computer Hardware; Computer Software; Business...,nan,nan,j13251
267,Data Incident Analysis Specialist - Nationwide,Job descriptionWe need:• Data Incident Analysi...,[],"Swindon Head Office, SW","Part Time, Permanent",Today,Other/Not Classified,nan,nan,mp25410ct1240-r002801427


<bound method IndexOpsMixin.value_counts of 0          £30,000.00 - £60,000.00  per year   Benefits:
1      £20,000.00 - £22,000.00  per year£10,000 Bonus...
2                                                     []
3                      £20,000.00 - £22,000.00  per year
4      £53,000.00 - £58,000.00  per yearExcellent Ben...
                             ...                        
264            £17,550.00 - £17,550.00  per yearbenefits
265                    £50,000.00 - £55,000.00  per year
266        £41,000.00 - £47,000.00  per year   Benefits:
267                                                   []
268                                                   []
Name: salary, Length: 269, dtype: object>

KeyError: 'job_title/company'